In [1]:
import torch
print(f"torch version {torch.__version__}")
from torch import nn
import datetime

# Importing TensorBoard libraries for visuvalization
from torch.utils.tensorboard import SummaryWriter

# Importing Ray Tune libraries for hyperparameter tuning
from ray import train, tune

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device
print(f"Device = {device}")

torch version 1.13.1
Device = cpu


In [2]:
X = torch.tensor([[1.], [2.], [3.], [4.], [5.], [6.], [7.], [8.], [9.], [10.]])
y = torch.tensor([[2.], [4.], [6.], [8.], [10.], [12.], [14.], [16.], [18.], [20.]])
#X_test = torch.tensor([[2.5], [22.], [-6.], [.25],])
#y_test = torch.tensor([[5.], [44.], [-12.], [.5],])
X_test = torch.tensor([[40.]])
y_test = torch.tensor([[80.]])
print(X.size())
print(y.size())
print(X_test.size())
print(y_test.size())

tboard_dir = "runs/pytoch_learning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S");
print(tboard_dir)
writer = SummaryWriter(tboard_dir)

torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([1, 1])
torch.Size([1, 1])
runs/pytoch_learning/20231024-200502


In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, X, y, transform=None, target_transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        return self.X[idx], self.y[idx]

In [4]:
full_dataset = MyDataset(X,y)
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [8, 2])

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Linear(1, 1)
        
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits
    
model = NeuralNetwork()
#print(f"Model structure : {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_stack.weight | Size: torch.Size([1, 1]) | Values : tensor([[0.5638]], grad_fn=<SliceBackward0>) 

Layer: linear_stack.bias | Size: torch.Size([1]) | Values : tensor([0.2933], grad_fn=<SliceBackward0>) 



In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        #print(f"Batch = {batch}")
        #print(f"X = {X}")
        #print(f"y = {y}")
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        # parameters after optimization
        #print("**** parms after optimizer ****")
        #for name, param in model.named_parameters():
        #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
        #print("\n")
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        return loss

In [8]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_loss

In [9]:
#learning_rate = 1e-3
learning_rate = 0.01
batch_size = 2
epochs = 1000

# Initialize the loss function
loss_fn = nn.MSELoss()
print(f"Loss function : {loss_fn} \n")

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(f"Optimizer : {optimizer}")

Loss function : MSELoss() 

Optimizer : SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
parms = []
weight = torch.empty((1,1))
bias = torch.empty((1,1))

for t in range (epochs):
    
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    
    for name, param in model.named_parameters():
    #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
        parms.append(param[:2])
    
    test_loss = test_loop(test_dataloader, model, loss_fn)

    weight = parms[0]
    bias = parms[1]
    
    print(f"Epoch {t+1}, Weight = {weight.detach().numpy()[0][0]}, Bias = {bias.detach().numpy()[0]}, Training Loss = {train_loss}, Testing Loss = {test_loss}")
    
    writer.add_scalars('Training vs. Test Loss',
                            {'Training' : train_loss, 'Test' : test_loss,}, t)
    
    writer.flush()

print("\n***** End of processing *****")

Epoch 1, Weight = 1.8211802244186401, Bias = 0.45981237292289734, Training Loss = 87.85010528564453, Testing Loss = 0.035569079453125596
Epoch 2, Weight = 1.9269405603408813, Bias = 0.4720745086669922, Training Loss = 0.6636887788772583, Testing Loss = 0.04814881458878517
Epoch 3, Weight = 1.9360451698303223, Bias = 0.4714001417160034, Training Loss = 0.049176041036844254, Testing Loss = 0.0623076930642128
Epoch 4, Weight = 1.9370365142822266, Bias = 0.46964672207832336, Training Loss = 0.04449833929538727, Testing Loss = 0.06312888860702515
Epoch 5, Weight = 1.93734610080719, Bias = 0.46780940890312195, Training Loss = 0.04411894828081131, Testing Loss = 0.06274387612938881
Epoch 6, Weight = 1.937597393989563, Bias = 0.4659716784954071, Training Loss = 0.043772630393505096, Testing Loss = 0.06226067245006561
Epoch 7, Weight = 1.9378430843353271, Bias = 0.46414056420326233, Training Loss = 0.04342930018901825, Testing Loss = 0.06177313253283501
Epoch 8, Weight = 1.9380874633789062, Bia